In [2]:
# Dependencies
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import *
import json
import requests
from pprint import pprint
from config import driver, username, password, host, port, database
from sqlalchemy import create_engine

In [3]:
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

In [4]:
contentChoiceDF = pd.read_sql_table('contentChoice', connection)
contentChoiceDF

,id,name,shortName,latitude,longitude,homePage
0,1,"University of California, Los Angeles",UCLA,34.079560,-118.444940,Y
1,2,"University of California, Berkeley",UCB,37.871853,-122.258423,N
2,3,"University of California, San Diego",UCSD,32.921990,-117.236560,N
3,4,"University of California, Santa Barbara",UCSB,34.413963,-119.848946,N
4,5,"University of California, Irvine",UCI,33.640990,-117.844370,N


In [2]:
city = "Los Angeles"
lat = 34.0522
lon = -118.2437

In [3]:
## Get initial endpoint for city
# Assign initial api url to a variable
lat_lon_url = f"https://api.weather.gov/points/{lat},{lon}"

# API call to retrieve enpoints for location of lat and lon
response = requests.get(lat_lon_url).json()

In [4]:
pprint(response)

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
              {'@version': '1.1',
               '@vocab': 'https://api.weather.gov/ontology#',
               'bearing': {'@type': 's:QuantitativeValue'},
               'city': 's:addressLocality',
               'county': {'@type': '@id'},
               'distance': {'@id': 's:Distance',
                            '@type': 's:QuantitativeValue'},
               'forecastGridData': {'@type': '@id'},
               'forecastOffice': {'@type': '@id'},
               'geo': 'http://www.opengis.net/ont/geosparql#',
               'geometry': {'@id': 's:GeoCoordinates',
                            '@type': 'geo:wktLiteral'},
               'publicZone': {'@type': '@id'},
               's': 'https://schema.org/',
               'state': 's:addressRegion',
               'unit': 'http://codes.wmo.int/common/unit/',
               'unitCode': {'@id': 's:unitCode', '@type': '@id'},
               'value': {'@id': 's:valu

In [5]:
# Specify the URL
# Major cities in California
weekly_url = response["properties"]["forecast"]
hourly_url = response["properties"]["forecastHourly"]
tz =  response["properties"]["timeZone"]

In [6]:
# Make request and store response
weekly_response = requests.get(weekly_url).json()

In [7]:
# Find generatedAt string; convert to datetime object
call_datetime_str = weekly_response["properties"]["generatedAt"]
call_datetime = parse(call_datetime_str)

In [8]:
# Find period forecasts; assign to variable
period_forecasts = weekly_response["properties"]["periods"]

In [9]:
weekly_forecast = []
for i, period_forecast in enumerate(period_forecasts):
    # Isolate date and time from StartTime and EndTime; convert to datetime object
    start_date = datetime.strptime(period_forecasts[i]["startTime"].split("T")[0],"%Y-%m-%d").date()
    start_time = datetime.strptime(period_forecasts[i]["startTime"].split("T")[1],"%H:%M:%S%z").time()
    end_date = datetime.strptime(period_forecasts[i]["endTime"].split("T")[0],"%Y-%m-%d").date()
    end_time = datetime.strptime(period_forecasts[i]["endTime"].split("T")[1],"%H:%M:%S%z").time()
    
    # Get min, max wind speeds; get wind speed units
    wind_speed = period_forecasts[i]["windSpeed"].split(" ")
    if len(wind_speed) == 2:
        min_wind_speed = float(wind_speed[0])
        max_wind_speed = np.nan
        wind_speed_unit = wind_speed[1]
    elif len(wind_speed) == 4:
        min_wind_speed = float(wind_speed[0])
        max_wind_speed = float(wind_speed[2])
        wind_speed_unit = wind_speed[3]

    # Append dates and times to period_forecast dictionary
    period_forecast["city"] = city
    period_forecast["startDate"] = start_date
    period_forecast["start_time"] = start_time
    period_forecast["endDate"] = end_date
    period_forecast["end_time"] = end_time
    period_forecast["minWindSpeed"] = min_wind_speed
    period_forecast["maxWindSpeed"] = max_wind_speed
    period_forecast["windSpeedUnit"] = wind_speed_unit
    period_forecast["latitude"] = lat
    period_forecast["longitude"] = lon
    period_forecast["retrievalDateTime"] = call_datetime
    
    # Append period_forcast to weekly_forecast list
    weekly_forecast.append(period_forecast)

# Create dataframe from "weekly_forecast" list
weekly_forecast_raw_df = pd.DataFrame(weekly_forecast)

In [10]:
# Copy weekly_forecast_raw_df to working dataframe
weekly_forecast_working_df = weekly_forecast_raw_df

# Convert startTime from string to datetime object
weekly_forecast_working_df["startTime"] = pd.to_datetime(weekly_forecast_working_df["startTime"])

# Convert endtTime from string to datetime object
weekly_forecast_working_df["endTime"] = pd.to_datetime(weekly_forecast_working_df["endTime"])

# Convert isDayTime from string to boolean
weekly_forecast_working_df.replace({"isDaytime": {"True": True, "False": False}}, inplace=True)

# Reasign column names
weekly_forecast_working_df.rename(columns={"startTime":"startDateTime", "endTime":"endDateTime", "start_time":"startTime", "end_time":"endTime", "number":"responseNumber", "name":"responseName"}, inplace=True)

# Rearange columns of dataframe
weekly_forecast_working_df = weekly_forecast_working_df[['responseNumber', 'responseName', 'city', 'latitude', 'longitude','startDateTime', 'startDate', 'startTime', 'endDateTime', 'endDate', 'endTime', 'isDaytime', 'temperature', 'temperatureUnit', 'temperatureTrend', 'windSpeed','minWindSpeed', 'maxWindSpeed', 'windSpeedUnit', 'windDirection', 'icon', 'shortForecast', 'detailedForecast', 'retrievalDateTime']]

In [11]:
# Copy working dataframe to final dataframe
weekly_forecast_df = weekly_forecast_working_df

In [12]:
# Make hourly forecast request and store response
hourly_response = requests.get(hourly_url).json()

In [13]:
# Find generatedAt string; convert to datetime object
request_datetime_str = hourly_response["properties"]["generatedAt"]
request_datetime = parse(call_datetime_str)

In [14]:
# Find hour forecasts; assign to variable
hour_forecasts = hourly_response["properties"]["periods"]

In [15]:
hourly_forecast = []
for i, hour_forecast in enumerate(hour_forecasts):
    # Isolate date and time from StartTime and EndTime; convert to datetime object
    start_date = datetime.strptime(hour_forecasts[i]["startTime"].split("T")[0],"%Y-%m-%d").date()
    start_time = datetime.strptime(hour_forecasts[i]["startTime"].split("T")[1],"%H:%M:%S%z").time()
    end_date = datetime.strptime(hour_forecasts[i]["endTime"].split("T")[0],"%Y-%m-%d").date()
    end_time = datetime.strptime(hour_forecasts[i]["endTime"].split("T")[1],"%H:%M:%S%z").time()
    
    # Get hour wind speed; get wind speed units
    wind_speed = hour_forecasts[i]["windSpeed"].split(" ")
    hour_wind_speed = float(wind_speed[0])
    wind_speed_unit = wind_speed[1]

    # Append dates and times to hour_forecast dictionary
    hour_forecast["city"] = city
    hour_forecast["startDate"] = start_date
    hour_forecast["start_time"] = start_time
    hour_forecast["endDate"] = end_date
    hour_forecast["end_time"] = end_time
    hour_forecast["hourWindSpeed"] = hour_wind_speed
    hour_forecast["windSpeedUnit"] = wind_speed_unit
    hour_forecast["latitude"] = lat
    hour_forecast["longitude"] = lon
    hour_forecast["retrievalDateTime"] = request_datetime
    
    # Append period_forcast to hourly_forecast list
    hourly_forecast.append(hour_forecast)

# Create dataframe from "hourly_forecast" list
hourly_forecast_raw_df = pd.DataFrame(hourly_forecast)

In [16]:
# Copy hourly_forecast_raw_df to working dataframe
hourly_forecast_working_df = hourly_forecast_raw_df

# Convert startTime from string to datetime object
hourly_forecast_working_df["startTime"] = pd.to_datetime(hourly_forecast_working_df["startTime"])

# Convert endtTime from string to datetime object
hourly_forecast_working_df["endTime"] = pd.to_datetime(hourly_forecast_working_df["endTime"])

# Convert isDayTime from string to boolean
hourly_forecast_working_df.replace({"isDaytime": {"True": True, "False": False}}, inplace=True)

# Reasign column names
hourly_forecast_working_df.rename(columns={"startTime":"startDateTime", "endTime":"endDateTime", "start_time":"startTime", "end_time":"endTime", "number":"responseNumber"}, inplace=True)

# Drop empty columns "name" and "detailedForecast"
hourly_forecast_working_df.drop(columns=["name", "detailedForecast"], inplace=True)

# Rearange columns of dataframe
hourly_forecast_working_df = hourly_forecast_working_df[['responseNumber', 'city', 'latitude', 'longitude','startDateTime', 'startDate', 'startTime', 'endDateTime', 'endDate', 'endTime', 'isDaytime', 'temperature', 'temperatureUnit', 'temperatureTrend', 'windSpeed','hourWindSpeed', 'windSpeedUnit', 'windDirection', 'icon', 'shortForecast', 'retrievalDateTime']]

In [17]:
# Copy working dataframe to final dataframe
hourly_forecast_df = hourly_forecast_working_df

In [18]:
hourly_forecast_df

,responseNumber,city,latitude,longitude,startDateTime,startDate,startTime,endDateTime,endDate,endTime,...,temperature,temperatureUnit,temperatureTrend,windSpeed,hourWindSpeed,windSpeedUnit,windDirection,icon,shortForecast,retrievalDateTime
0,1,Los Angeles,34.0522,-118.2437,2021-10-16 10:00:00-07:00,2021-10-16,10:00:00,2021-10-16 11:00:00-07:00,2021-10-16,11:00:00,...,79,F,None,10 mph,10.0,mph,ESE,https://api.weather.gov/icons/land/day/few?siz...,Sunny,2021-10-16 12:39:15+00:00
1,2,Los Angeles,34.0522,-118.2437,2021-10-16 11:00:00-07:00,2021-10-16,11:00:00,2021-10-16 12:00:00-07:00,2021-10-16,12:00:00,...,84,F,None,10 mph,10.0,mph,ENE,https://api.weather.gov/icons/land/day/few?siz...,Sunny,2021-10-16 12:39:15+00:00
2,3,Los Angeles,34.0522,-118.2437,2021-10-16 12:00:00-07:00,2021-10-16,12:00:00,2021-10-16 13:00:00-07:00,2021-10-16,13:00:00,...,87,F,None,10 mph,10.0,mph,ENE,https://api.weather.gov/icons/land/day/few?siz...,Sunny,2021-10-16 12:39:15+00:00
3,4,Los Angeles,34.0522,-118.2437,2021-10-16 13:00:00-07:00,2021-10-16,13:00:00,2021-10-16 14:00:00-07:00,2021-10-16,14:00:00,...,90,F,None,10 mph,10.0,mph,ENE,https://api.weather.gov/icons/land/day/few?siz...,Sunny,2021-10-16 12:39:15+00:00
4,5,Los Angeles,34.0522,-118.2437,2021-10-16 14:00:00-07:00,2021-10-16,14:00:00,2021-10-16 15:00:00-07:00,2021-10-16,15:00:00,...,91,F,None,5 mph,5.0,mph,WSW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,2021-10-16 12:39:15+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,152,Los Angeles,34.0522,-118.2437,2021-10-22 17:00:00-07:00,2021-10-22,17:00:00,2021-10-22 18:00:00-07:00,2021-10-22,18:00:00,...,73,F,None,10 mph,10.0,mph,WSW,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,2021-10-16 12:39:15+00:00
152,153,Los Angeles,34.0522,-118.2437,2021-10-22 18:00:00-07:00,2021-10-22,18:00:00,2021-10-22 19:00:00-07:00,2021-10-22,19:00:00,...,69,F,None,10 mph,10.0,mph,WSW,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,2021-10-16 12:39:15+00:00
153,154,Los Angeles,34.0522,-118.2437,2021-10-22 19:00:00-07:00,2021-10-22,19:00:00,2021-10-22 20:00:00-07:00,2021-10-22,20:00:00,...,65,F,None,10 mph,10.0,mph,WSW,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,2021-10-16 12:39:15+00:00
154,155,Los Angeles,34.0522,-118.2437,2021-10-22 20:00:00-07:00,2021-10-22,20:00:00,2021-10-22 21:00:00-07:00,2021-10-22,21:00:00,...,63,F,None,5 mph,5.0,mph,SW,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,2021-10-16 12:39:15+00:00


In [19]:
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

In [20]:
weekly_forecast_df.to_sql('dailyForecastTB',connection, if_exists='append', index=False)

In [21]:
# hourly_forecast_df.to_sql('hourlyForecastTB',connection, if_exists='append', index=False)

In [23]:
historicDailyForecastDF = pd.read_sql_table('dailyForecastTB', connection)
historicDailyForecastDF

,id,responseNumber,responseName,city,latitude,longitude,startDateTime,startDate,startTime,endDateTime,...,temperatureTrend,windSpeed,minWindSpeed,maxWindSpeed,windSpeedUnit,windDirection,icon,shortForecast,detailedForecast,retrievalDateTime
0,2,1,Overnight,Los Angeles,34.0522,-118.2437,2021-10-16 12:00:00,2021-10-16,05:00:00,2021-10-16 13:00:00,...,None,5 mph,5.0,NaN,mph,NNE,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 62. North nort...",2021-10-16 12:39:15
1,3,2,Saturday,Los Angeles,34.0522,-118.2437,2021-10-16 13:00:00,2021-10-16,06:00:00,2021-10-17 01:00:00,...,None,5 to 10 mph,5.0,10.0,mph,ENE,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 91. East northeast win...",2021-10-16 12:39:15
2,4,3,Saturday Night,Los Angeles,34.0522,-118.2437,2021-10-17 01:00:00,2021-10-16,18:00:00,2021-10-17 13:00:00,...,None,0 to 5 mph,0.0,5.0,mph,SSE,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,"Partly cloudy, with a low around 58. South sou...",2021-10-16 12:39:15
3,5,4,Sunday,Los Angeles,34.0522,-118.2437,2021-10-17 13:00:00,2021-10-17,06:00:00,2021-10-18 01:00:00,...,None,0 to 5 mph,0.0,5.0,mph,SSE,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny, with a high near 76. South south...",2021-10-16 12:39:15
4,6,5,Sunday Night,Los Angeles,34.0522,-118.2437,2021-10-18 01:00:00,2021-10-17,18:00:00,2021-10-18 13:00:00,...,None,0 to 5 mph,0.0,5.0,mph,SSE,https://api.weather.gov/icons/land/night/rain?...,Patchy Drizzle,Patchy drizzle and patchy fog after 11pm. Most...,2021-10-16 12:39:15
5,7,6,Monday,Los Angeles,34.0522,-118.2437,2021-10-18 13:00:00,2021-10-18,06:00:00,2021-10-19 01:00:00,...,None,5 to 10 mph,5.0,10.0,mph,S,https://api.weather.gov/icons/land/day/rain/bk...,Patchy Drizzle then Mostly Cloudy,Patchy drizzle and patchy fog before 11am. Mos...,2021-10-16 12:39:15
6,8,7,Monday Night,Los Angeles,34.0522,-118.2437,2021-10-19 01:00:00,2021-10-18,18:00:00,2021-10-19 13:00:00,...,None,5 to 10 mph,5.0,10.0,mph,SE,https://api.weather.gov/icons/land/night/fog?s...,Patchy Fog,"Patchy fog after 11pm. Mostly cloudy, with a l...",2021-10-16 12:39:15
7,9,8,Tuesday,Los Angeles,34.0522,-118.2437,2021-10-19 13:00:00,2021-10-19,06:00:00,2021-10-20 01:00:00,...,None,5 to 10 mph,5.0,10.0,mph,SSE,https://api.weather.gov/icons/land/day/fog/sct...,Patchy Fog then Mostly Sunny,"Patchy fog before 11am. Mostly sunny, with a h...",2021-10-16 12:39:15
8,10,9,Tuesday Night,Los Angeles,34.0522,-118.2437,2021-10-20 01:00:00,2021-10-19,18:00:00,2021-10-20 13:00:00,...,None,0 to 10 mph,0.0,10.0,mph,WNW,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,"Partly cloudy, with a low around 52.",2021-10-16 12:39:15
9,11,10,Wednesday,Los Angeles,34.0522,-118.2437,2021-10-20 13:00:00,2021-10-20,06:00:00,2021-10-21 01:00:00,...,None,5 to 10 mph,5.0,10.0,mph,W,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny, with a high near 72.",2021-10-16 12:39:15
